<a href="https://colab.research.google.com/github/dotsnangles/Transformers-Text-Summarisation/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets transformers rouge-score nltk sentencepiece

     |████████████████████████████████| 346 kB 4.2 MB/s 
     |████████████████████████████████| 4.2 MB 88.7 MB/s 
     |████████████████████████████████| 1.2 MB 86.9 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
     |████████████████████████████████| 212 kB 92.8 MB/s 
     |████████████████████████████████| 1.1 MB 84.4 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 140 kB 95.0 MB/s 
     |████████████████████████████████| 596 kB 61.0 MB/s 
     |████████████████████████████████| 127 kB 95.0 MB/s 
     |████████████████████████████████| 6.6 MB 56.6 MB/s 
     |████████████████████████████████| 271 kB 81.9 MB/s 
     |████████████████████████████████| 144 kB 92.3 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 112 kB 54.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source o

In [2]:
!gdown -q 1fGGvQ_NuaBNitPZwK4gAhCAuz8uAf3DX
!gdown -q 1BPI0rGQ1LseACn8kPVmaKreiU3VBgoxh

In [46]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from datasets import load_dataset, load_metric, Dataset
import pandas as pd
import numpy as np
import re
import nltk
import torch

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Load Data

In [5]:
train_path = '/content/train_df.csv'
test_path = '/content/test_df.csv'

In [6]:
train_df = pd.read_csv(train_path, index_col=False)
# train_df = train_df[['context', 'summary']]

### Load Model & Tokenizer & Metric

In [7]:
# model_name = 'csebuetnlp/mT5_multilingual_XLSum'
model_name = 'google/mt5-small'
metric_name = 'rouge'

In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
metric = load_metric(metric_name)

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


### Declear Functions

In [9]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["context"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

### Training

In [11]:
train_df.drop(['Unnamed: 0', 'id', 'title', 'region', 'agenda', 'total'], axis=1, inplace=True)

In [12]:
dataset = Dataset.from_pandas(train_df)

In [13]:
dataset = dataset.train_test_split(0.2, seed=100)

In [14]:
prefix = ""
# prefix = "summarize: "

In [15]:
dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

In [52]:
batch_size = 4
num_train_epochs = 6
es = EarlyStoppingCallback(early_stopping_patience=3)

In [55]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./log",
    num_train_epochs=num_train_epochs,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    # lr_scheduler_type='linear',
    # warmup_ratio=0.1,
    save_strategy='epoch',
    save_total_limit=3,
    # fp16=True,
    predict_with_generate=True,
    # generation_max_length=,
    # generation_num_beams=,
    metric_for_best_model='eval_loss',
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[es],
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [56]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: summary, context. If summary, context are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2395
  Num Epochs = 6
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3594


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.896900,1.199623,19.416200,12.580000,19.327300,19.413000,13.966600
2,1.636500,1.106397,45.643500,27.488200,45.313500,45.549800,18.642700
3,1.542900,1.071624,46.182400,28.667200,45.837700,45.968800,18.687800
4,1.508000,1.057907,46.504600,28.739500,46.132100,46.262700,18.716200
5,1.384200,1.043865,46.478900,28.625500,46.070200,46.252900,18.709500
6,1.339000,1.041468,46.563500,28.625500,46.164500,46.361900,18.727900


The following columns in the evaluation set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: summary, context. If summary, context are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 599
  Batch size = 4
Saving model checkpoint to ./log/checkpoint-599
Configuration saved in ./log/checkpoint-599/config.json
Model weights saved in ./log/checkpoint-599/pytorch_model.bin
tokenizer config file saved in ./log/checkpoint-599/tokenizer_config.json
Special tokens file saved in ./log/checkpoint-599/special_tokens_map.json
Copy vocab file to ./log/checkpoint-599/spiece.model
The following columns in the evaluation set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: summary, context. If summary, context are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Ru

TrainOutput(global_step=3594, training_loss=1.5259484751196657, metrics={'train_runtime': 2596.2724, 'train_samples_per_second': 5.535, 'train_steps_per_second': 1.384, 'total_flos': 1.377827614166016e+16, 'train_loss': 1.5259484751196657, 'epoch': 6.0})

In [57]:
preds = trainer.predict(dataset["test"])

The following columns in the test set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: summary, context. If summary, context are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 599
  Batch size = 4


In [58]:
preds.metrics

{'test_gen_len': 18.7279,
 'test_loss': 1.0414682626724243,
 'test_rouge1': 46.5635,
 'test_rouge2': 28.6255,
 'test_rougeL': 46.1645,
 'test_rougeLsum': 46.3619,
 'test_runtime': 82.8659,
 'test_samples_per_second': 7.229,
 'test_steps_per_second': 1.81}

In [64]:
dataset['test'][0].keys()

dict_keys(['context', 'summary', 'input_ids', 'attention_mask', 'labels'])

In [48]:
# input_ = dataset['test'][0]['input_ids']
# tokenizer.decode(input_, skip_special_tokens=True)

In [65]:
for data, pred in zip(dataset['test'], preds.predictions):
    context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)
    summary = tokenizer.decode(data['labels'], skip_special_tokens=True)
    pred = tokenizer.decode(pred[2:], skip_special_tokens=True)
    print(f'입력: {context}')
    print(f'정답: {summary}')
    print(f'예측: {pred}\n')

입력: 시설관리사업소장입니다. 79페이지입니다. 2019년도 행정사무감사 시정ᆞ건의사항 조치결과에 대해서 보고드리겠습니다. 시설관리사업소 소관은 건의사항 중 공통사항은 없으며, 부서 2건이 되겠습니다. 먼저 최용락 의원님께서 삼성체육공원 그라운드골프장을 빠른 시일 내에 착공할 수 있도록 하라고 건의하신 사항은 2019년 9월에 공원조성계획 변경을 완료하였고, 2020년 2월에 균형발전특별회계 사업으로 총사업비 40억원을 신청하였습니다. 현재 지방재정투자심사 승인을 위한 서면 심의 중에 있으나 금일 심의ᆞ의결 통보를 받았습니다. 1회 추경에 실시설계비 1억원을 편성하여 착수 후 2021년 6월까지 실시계획인가를 득하고, 2021년 7월에 공사 착공하여 2022년도에 준공하고자 합니다. 국비 확보를 위해 최선을 다하겠으며 계획을 차질 없이 진행하여 빠른 시일 내에 공사착공할 수 있도록 적극 노력하겠습니다.다음은 서효석 의원님께서 금왕생활체육공원 공실에 대하여 활용방안을 찾아보고 물품 보관 여건이 된다면 자치행정과와 협의하여 해당 봉사단체에서 활용할 수 있도록 조치를 취하라고 건의하신 사항은 봉사단체와 시설관리사업소 직원이 금왕생활체육공원 라커룸을 현지확인하였으나 라커룸은 창고시설로서 규모가 21m2, 약 6.3평 정도로 봉사단체에서 요청하는 사무실 및 물품 보관 용도로는 규모가 작아 부적합함을 상호 확인하였습니다. 현재 라커 8호에 대해서는 한우협회에서 사용하지 않아 음성군에서 산불진압대 대기실로 이용하고 있습니다. 추후 자치행정과 대외협력팀에서 봉사단체 사무실을 별도로 마련할 예정에 있음을 말씀드립니다.이상으로 시설관리사업소 소관 보고를 마치겠습니다.보고를 들으시고 질의하실 의원님 계시면 질의하여 주시기 바랍니다.서효석 의원님 질의하여 주시기 바랍니다.소장님 자료 준비하고 설명하시느냐고 수고하십니다. 특히 건의드린 건에 대해서 발빠르게 처리하고 조치하시는 부분에 대해서 감사하다는 말씀을 드리고요. 본 의원이 생활체육공원 내에 공실을 파악을 해달라는 부분도 면밀히

In [ ]:
# del trainer, model
# torch.cuda.empty_cache()

### Test Model

In [ ]:
# model_name = 'csebuetnlp/mT5_multilingual_XLSum'
# model_name = 'google/mt5-small'
# metric_name = 'rouge'

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# metric = load_metric(metric_name)

In [ ]:
article_text = train_df['context'].iloc[0]
article_text

'의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제1차 본회의 개의를 선포합니다. 먼저 의사팀장으로부터 의회 관련 사항에 대한 보고가 있겠습니다. 의사팀장은 보고하여 주시기 바랍니다.의사팀장 이은미 입니다. 의회관련 사항을 보고 드리겠습니다. 먼저 제207회 완주군의회 임시회 소집경위를 보고 드리겠습니다. 지방자치법 제45조 제2항의 규정에 의거 서남용 의원님 외 4인의 의원님으로부터 임시회 집회 요구가 있어 지난 8월6일 집회 공고를 하고 오늘 집회를 갖게 되었습니다. 다음은 의안접수 및 회부사항입니다. 먼저 의원발의 안건입니다. 완주군의회 의원 의정활동비등 지급에 관한 조례 일부개정 조례안, 완주군 향토문화유산 보호 및 관리 조례안, 완주군 싸움소 육성 지원에 관한 조례안 등 총 3건으로, 해당 상임의원회로 회부 하였습니다. 다음은 완주군수로부터 제출된 안건입니다. 삼례문화예술촌 설치 및 운영조례 일부개정 조례안 등 총 40건으로 해당 상임의원회로 회부하였습니다. 자세한 내용은 배부해 드린 유인물을 참조하여 주시기 바랍니다. 이상 보고를 마치겠습니다'

In [ ]:
with tokenizer.as_target_tokenizer():
        labels = tokenizer(article_text, max_length=128, truncation=True)
labels

{'input_ids': [259, 649, 10231, 611, 6104, 38714, 7192, 3919, 97030, 5694, 44903, 11267, 260, 14922, 78591, 143048, 20282, 55163, 6259, 42556, 6788, 259, 124062, 13672, 164491, 26421, 1601, 6788, 6259, 353, 5913, 17245, 49936, 259, 56282, 1023, 10563, 10982, 5699, 260, 259, 27117, 8115, 259, 91758, 27373, 1870, 139641, 259, 164491, 259, 19486, 259, 40895, 873, 9153, 259, 32477, 929, 3083, 95226, 260, 259, 91758, 27373, 59438, 259, 32477, 7192, 3919, 97030, 5694, 44903, 11267, 260, 91758, 27373, 1870, 1602, 869, 4532, 259, 5068, 260, 259, 164491, 19486, 259, 40895, 611, 259, 32477, 259, 84336, 95226, 260, 259, 27117, 8115, 6259, 42556, 6788, 259, 124062, 13672, 164491, 26421, 1601, 6788, 6345, 13022, 7108, 4924, 1023, 259, 32477, 259, 84336, 95226, 260, 90674, 2565, 3493, 8280, 6259, 3300, 5268, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
labels = tokenizer(article_text, max_length=128, truncation=True)
labels

{'input_ids': [259, 649, 10231, 611, 6104, 38714, 7192, 3919, 97030, 5694, 44903, 11267, 260, 14922, 78591, 143048, 20282, 55163, 6259, 42556, 6788, 259, 124062, 13672, 164491, 26421, 1601, 6788, 6259, 353, 5913, 17245, 49936, 259, 56282, 1023, 10563, 10982, 5699, 260, 259, 27117, 8115, 259, 91758, 27373, 1870, 139641, 259, 164491, 259, 19486, 259, 40895, 873, 9153, 259, 32477, 929, 3083, 95226, 260, 259, 91758, 27373, 59438, 259, 32477, 7192, 3919, 97030, 5694, 44903, 11267, 260, 91758, 27373, 1870, 1602, 869, 4532, 259, 5068, 260, 259, 164491, 19486, 259, 40895, 611, 259, 32477, 259, 84336, 95226, 260, 259, 27117, 8115, 6259, 42556, 6788, 259, 124062, 13672, 164491, 26421, 1601, 6788, 6345, 13022, 7108, 4924, 1023, 259, 32477, 259, 84336, 95226, 260, 90674, 2565, 3493, 8280, 6259, 3300, 5268, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
encoded = tokenizer.encode(article_text)
# encoded

In [ ]:
decoded = list(map(tokenizer.decode, encoded))
len(decoded)

371

In [ ]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

In [ ]:
tokenizer.max_model_input_sizes

{'t5-11b': 512, 't5-3b': 512, 't5-base': 512, 't5-large': 512, 't5-small': 512}